In [1]:
from tensorflow import keras

import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

from keras.datasets import mnist
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import MaxPool2D, Dense, Dropout, Conv2D ,Flatten 
from keras.utils import np_utils
import cv2

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

def resize(img_array):
    arr = np.empty((img_array.shape[0], 28, 28))

    for i in range(len(img_array)):
        img = img_array[i].reshape(28, 28).astype('uint8')
        img = cv2.resize(img, (28, 28))
        img = img.astype('float32')/255
        arr[i] = img
        
    return arr


x_train = resize(x_train)
x_test = resize(x_test)
x_train = np.stack((x_train,), axis=-1)
x_test = np.stack((x_test,), axis=-1)
y_train = to_categorical(y_train) 

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=2020)
x_train.shape

11493376/11490434 [==============================] - 0s 0us/step


(48000, 28, 28, 1)

In [3]:

model=Sequential()
model.add(Conv2D(128,(3,3),activation='relu',input_shape=(28,28,1)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(10,activation='softmax'))



In [4]:
INIT_LR = 1e-3
EPOCHS = 10
BS = 32
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
# train the head of the network

model.fit(x_train, y_train, validation_data=(x_test, y_test), 
          epochs=10, batch_size=BS)



Epoch 1/10
1500/1500 [==============================] - 121s 80ms/step - loss: 0.1934 - accuracy: 0.9407 - val_loss: 0.0617 - val_accuracy: 0.9794
Epoch 2/10
1500/1500 [==============================] - 119s 79ms/step - loss: 0.0688 - accuracy: 0.9796 - val_loss: 0.0465 - val_accuracy: 0.9857
Epoch 3/10
1500/1500 [==============================] - 120s 80ms/step - loss: 0.0523 - accuracy: 0.9841 - val_loss: 0.0466 - val_accuracy: 0.9872
Epoch 4/10
1500/1500 [==============================] - 120s 80ms/step - loss: 0.0384 - accuracy: 0.9885 - val_loss: 0.0343 - val_accuracy: 0.9901
Epoch 5/10
1500/1500 [==============================] - 119s 80ms/step - loss: 0.0324 - accuracy: 0.9900 - val_loss: 0.0338 - val_accuracy: 0.9905
Epoch 6/10
1500/1500 [==============================] - 121s 81ms/step - loss: 0.0256 - accuracy: 0.9918 - val_loss: 0.0371 - val_accuracy: 0.9905
Epoch 7/10
1500/1500 [==============================] - 121s 80ms/step - loss: 0.0230 - accuracy: 0.9926 - val_loss: 0

In [ ]:
from sklearn.metrics import classification_report

class_to_label_map = {'zero' : 0, 'one' : 1, 'two' : 2, 'three': 3, "four": 4, "five": 5, "six": 6, "seven" : 7, "eight": 8, "nine": 9}

print("[INFO] evaluating network...")
predIdxs = model.predict(x_test, batch_size=BS)

predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(y_test.argmax(axis=1), predIdxs, target_names= class_to_label_map))

[INFO] evaluating network...
              precision    recall  f1-score   support

        zero       0.99      0.99      0.99      1148
         one       1.00      0.99      0.99      1380
         two       0.99      0.99      0.99      1184
       three       1.00      0.99      0.99      1252
        four       0.98      1.00      0.99      1191
        five       0.99      0.99      0.99      1093
         six       0.99      0.99      0.99      1148
       seven       0.99      0.99      0.99      1219
       eight       0.99      0.99      0.99      1166
        nine       0.99      0.98      0.98      1219

    accuracy                           0.99     12000
   macro avg       0.99      0.99      0.99     12000
weighted avg       0.99      0.99      0.99     12000



In [ ]:
# show the confusion matrix and accuracy

cm = confusion_matrix(y_test.argmax(axis=1), predIdxs)

total = sum(sum(cm))
s=0
for i in range(0,10):
  s=s+cm[i,i]

acc =  s/ total

print(cm)
print("acc: {:.4f}".format(acc))


[[1142    0    0    0    2    1    1    0    0    2]
 [   1 1371    1    0    1    0    0    5    1    0]
 [   2    0 1174    1    3    0    0    1    2    1]
 [   1    0    2 1235    0    5    1    2    2    4]
 [   0    1    0    0 1186    0    2    0    0    2]
 [   0    3    0    1    0 1082    2    1    3    1]
 [   4    0    1    0    1    0 1140    0    2    0]
 [   0    0    3    0    0    0    0 1210    1    5]
 [   1    1    2    1    0    0    2    0 1158    1]
 [   2    0    0    0   14    5    1    1    3 1193]]
acc: 0.9909


In [ ]:
# Save whole model for download
model.save("model.h5")
!pip install tensorflowjs

     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 112kB 14.2MB/s 
  Found existing installation: tensorflow-hub 0.10.0
    Uninstalling tensorflow-hub-0.10.0:
      Successfully uninstalled tensorflow-hub-0.10.0


In [ ]:
!tensorflowjs_converter --input_format keras '/content/model.h5' '/content/model'

2020-12-09 13:49:38.621173: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
